In [8]:
import json
import requests
import os
import pandas as pd
from neo4j import GraphDatabase
from data_normalizer import normalize
from dotenv import load_dotenv
import constants
import prettytable

load_dotenv()

C:\Users\ToanLe\AppData\Roaming\Python\Python37\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


False

In [9]:
def get_concepts(text):
    with open('data/intent_alias_data.json', encoding="utf8") as f:
        dictionary = json.load(f)

    out = {}
    for concept in dictionary:
        for alias in sorted(dictionary[concept], key=len, reverse=1):
            if alias in text:
                out[concept] = alias
                break
    return out

In [10]:
NER_MODEL_BERT = "phobert_large"
NER_MODEL_BILSTM = "BiLSTM"
NER_MODEL_BILSTM_CRF = "BiLSTM+CRF"

INTENT_MODEL_ONE_VS_REST = "onevsrest"


def extract_ner(text, model=NER_MODEL_BERT):
    """
    Input Arguments:
        - text : the sentence which will be extracted NER
    """
    ner_service_url = os.getenv("NER_SERVICE_URL",
                                default="http://localhost:8001/api/v1/ner")

    data = {'model': model, 'text': text}

    headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}

    r = requests.post(ner_service_url, data=json.dumps(data), headers=headers)

    return r.json()

In [11]:
def ner_postprocess(entities_raw):
    """Postprocess for NER: 
    1. normalize values
    2. only accept the related information with previous question

    Args:
        entities_raw (Dict): result of NER service

    Returns:
        Dict, Dict: raw and normalized entities
    """
    entities_normed = dict()
    entities_raw_out = dict()
    # print("entities_raw: ", entities_raw)
    for entity in entities_raw:
        key = entity['label']
        if key == 'O': continue

        value_raw = entity['content']
        if key not in entities_raw_out.keys():
            entities_raw_out[key] = [value_raw]
        else:
            entities_raw_out[key].append(value_raw)

        value_normed = normalize(value_raw, key)
        if key not in entities_normed.keys():
            entities_normed[key] = [value_normed]
        else:
            entities_normed[key].append(value_normed)

    return entities_raw_out, entities_normed

In [12]:
def get_entities(text):
    entities_raw = extract_ner(text)
    _, entities = ner_postprocess(entities_raw)
    return entities

In [19]:
def gen_query(concepts_keys, evidences_keys, targets, evidences):
    out = """
    MATCH {} 
    WHERE {}
    RETURN {}
    LIMIT 30
        """
    conditions = []
    matched_labels = []
    idx_target_labels = []
    condition_labels = []
    returned_labels = []

    # Define matched labels.
    if not concepts_keys:
        matched_labels = ['n']
    else:
        for idx, target in enumerate(concepts_keys):
            idx_target_labels.append('n' + str(idx))
            matched_labels.append('(n' + str(idx) + ':' + target + ')')

    # Define returned labels.
    returned_labels = matched_labels.copy()

    ##
    # Define condition labels.
    # #
    attrs = [constants.LABEL_REAL_ESTATE_TYPE, constants.LABEL_REAL_ESTATE_SUB_TYPE, \
            constants.LABEL_POSITION, constants.LABEL_DIRECTION, \
            constants.LABEL_FRONT_LENGTH, constants.LABEL_ROAD_WIDTH, \
            constants.LABEL_FLOOR, constants.LABEL_BED_ROOM, constants.LABEL_LIVING_ROOM, constants.LABEL_BATH_ROOM, \
            constants.LABEL_SURROUNDING, constants.LABEL_PROJECT_NAME, \
            constants.LABEL_LEGAL, constants.LABEL_TRANSACTION]

    key2dbcol = {
        'tang': constants.LABEL_FLOOR,
        'ban cong': constants.LABEL_FLOOR_BAN_CONG,
        'gac': constants.LABEL_FLOOR_GAC,
        'ham': constants.LABEL_FLOOR_HAM,
        'lung': constants.LABEL_FLOOR_LUNG,
        'san thuong': constants.LABEL_FLOOR_SAN_THUONG,
        'tret': constants.LABEL_FLOOR_TRET
    }

    for attr in attrs:
        if attr in evidences:
            if attr == constants.LABEL_FLOOR:
                for val in evidences[attr]:
                    target_k = key2dbcol[val['type']]
                    target_v = val['value']

                    for match_label in matched_labels:
                        if target_k in match_label:
                            idx_condition_node = match_label.split(
                                '(')[1].split(':')[0]
                            returned_labels.remove(match_label)

                    # conditions.append(f"{target_k} = '{target_v}'")
                    conditions.append(
                        f"{idx_condition_node}.individual = '{target_v}'")
            else:
                # if attr == constants.LABEL_REAL_ESTATE_TYPE or attr == constants.LABEL_REAL_ESTATE_SUB_TYPE:
                #     attr = "House"

                for match_label in matched_labels:
                    if attr in match_label:
                        idx_condition_node = match_label.split('(')[1].split(
                            ':')[0]
                        returned_labels.remove(match_label)

                # conditions.append(f"{attr}.individual = '{evidences[attr][0]}'")
                conditions.append(
                    f"{idx_condition_node}.individual = '{evidences[attr][0]}'"
                )

    ##
    # Condition for city, district, ward, street.
    # #
    loc_attrs = [
        constants.LABEL_DISTRICT, constants.LABEL_CITY, constants.LABEL_WARD,
        constants.LABEL_STREET
    ]

    for attr in loc_attrs:
        if attr in evidences:
            for match_label in matched_labels:
                if attr in match_label:
                    idx_condition_node = match_label.split('(')[1].split(
                        ':')[0]

                    returned_labels.remove(match_label)

            conditions.append(
                f"{idx_condition_node}.individual = '{evidences[attr][0]}'")

    ##
    # Condition for price.
    # #
    PRICE_OFFSET_CONST = 0.1
    # print("evidences-1: ", evidences)
    # print()
    # price_title = constants.LABEL_PRICE

    if constants.LABEL_PRICE in evidences:
        # print("evidences-2: ", evidences)

        for ele in evidences[constants.LABEL_PRICE][:1]:
            low, high = ele

            if high is None:
                high = low + low * PRICE_OFFSET_CONST
                low = low - low * PRICE_OFFSET_CONST

            for match_label in matched_labels:
                if constants.LABEL_PRICE in match_label:
                    idx_condition_node = match_label.split('(')[1].split(
                        ':')[0]

                    returned_labels.remove(match_label)

            # conditions.append(f"{constants.LABEL_PRICE} BETWEEN {low} AND {high}")
            conditions.append(
                f"{idx_condition_node}.individual = BETWEEN ' {low} ' AND ' {high} '"
            )

    ##
    # Condition for area.
    # #
    AREA_OFFSET_CONST = 0.1

    if constants.LABEL_AREA in evidences:
        for ele in evidences[constants.LABEL_AREA][:1]:
            low, high = ele

            if high is None:
                high = low + low * AREA_OFFSET_CONST
                low = low - low * AREA_OFFSET_CONST

            for match_label in matched_labels:
                if constants.LABEL_AREA in match_label:
                    idx_condition_node = match_label.split('(')[1].split(
                        ':')[0]

                    returned_labels.remove(match_label)

            # conditions.append(f"{constants.LABEL_AREA} BETWEEN {low} AND {high}")
            conditions.append(
                f"{idx_condition_node}.individual = BETWEEN ' {low} ' AND ' {high} '"
            )

    ##
    # Condition for usage.
    # #
    if constants.LABEL_USAGE in evidences:
        for match_label in matched_labels:
            if constants.LABEL_USAGE in match_label:
                idx_condition_node = match_label.split('(')[1].split(':')[0]

                returned_labels.remove(match_label)

        # conditions.append("({})".format(" OR ".join([f"{constants.LABEL_USAGE} LIKE '%, {x},%' + 'OR {constants.LABEL_USAGE} LIKE '{x}, %' OR {constants.LABEL_USAGE} LIKE '%, {x}'" for x in evidences[constants.LABEL_USAGE]])))
        conditions.append("({})".format(" OR ".join([
            f"{idx_condition_node}.individual LIKE '%, {x},%' + 'OR {idx_condition_node}.individual LIKE '{x}, %' OR {idx_condition_node}.individual LIKE '%, {x}'"
            for x in evidences[constants.LABEL_USAGE]
        ])))

    ##
    # Adjust returned labels
    # #
    adjusted_return_labels = []

    for return_label in returned_labels:
        return_label = f"{return_label.split('(')[1].split(':')[0]}.individual as {return_label.split(':')[1].split(')')[0]}"

        adjusted_return_labels.append(return_label)

    return out.format(
        ', '.join(matched_labels),
        ' AND '.join([f"{x}" for x in conditions]),
        ', '.join(adjusted_return_labels),
    )


In [14]:
def gen_query_ontology(text):
    table = prettytable.PrettyTable(["Step", "Result"])
    table.add_row(["Input", text])

    concepts = get_concepts(text)
    # print("concepts: ", concepts)

    evidences = get_entities(text)

    table.add_rows([["Match alias", concepts], ["Find individuals",
                                                evidences]])

    targets = list(set(concepts.keys()).difference(set(evidences.keys())))
    table.add_row(["Target concepts", targets])

    query = gen_query(concepts.keys(), evidences.keys(), targets, evidences)
    table.add_row(["Query", query])

    print(table)
    return query

In [20]:
# text = "nhà ở quận 1 thường có giá khoảng bao nhiêu"
text = "nhà hoặc căn hộ giá khoảng 2 tỷ thì mua ở quận nào"

cqlNodeQuery = gen_query_ontology(text)
# i = 0
# print(cqlNodeQuery)

entities_raw:  [{'content': 'nhà', 'label': 'real_estate_type'}, {'content': 'hoặc', 'label': 'O'}, {'content': 'căn hộ', 'label': 'real_estate_type'}, {'content': 'giá khoảng', 'label': 'O'}, {'content': '2 tỷ', 'label': 'price'}, {'content': 'thì', 'label': 'O'}, {'content': 'mua', 'label': 'transaction'}, {'content': 'ở quận nào', 'label': 'O'}]
evidences-2:  {'real_estate_type': ['nha', 'can ho'], 'price': [(2000000000.0, None)], 'transaction': ['mua']}
+------------------+-----------------------------------------------------------------------------------------------------------------------------+
|       Step       |                                                            Result                                                           |
+------------------+-----------------------------------------------------------------------------------------------------------------------------+
|      Input       |                                      nhà hoặc căn hộ giá khoảng 2 tỷ thì mua

In [9]:
# Connect neo4j DB.
driver = GraphDatabase.driver('bolt://localhost:7687',
                              auth=('neo4j', 'password'))


In [10]:
def run_query(query):
    with driver.session(database="htdb") as session:
        results = session.run(query)

        table_results = prettytable.PrettyTable(results.keys())
        for r in results:
            table_results.add_row(r.values())

        return table_results


In [11]:
result_data = run_query(cqlNodeQuery)

print(result_data)

+-----------+----------------+
|   House   |     Price      |
+-----------+----------------+
| shophouse |  1000000000.0  |
| shophouse | 22000000000.0  |
| shophouse |  5000000000.0  |
| shophouse |  4000000000.0  |
| shophouse | 130000000000.0 |
| shophouse |  3000000000.0  |
| shophouse |  6000000000.0  |
| shophouse |  560000000.0   |
| shophouse | 17000000000.0  |
| shophouse |  2000000000.0  |
| shophouse |  8000000000.0  |
| shophouse |  800000000.0   |
| shophouse |  9000000000.0  |
| shophouse | 14000000000.0  |
| shophouse |  320000000.0   |
| shophouse | 35000000000.0  |
| shophouse | 10000000000.0  |
| shophouse |  7000000000.0  |
| shophouse |  780000000.0   |
| shophouse | 15000000000.0  |
| shophouse |   30000000.0   |
| shophouse |  630000000.0   |
| shophouse | 19000000000.0  |
| shophouse |  550000000.0   |
| shophouse |   2000000.0    |
| shophouse |  670000000.0   |
| shophouse | 13000000000.0  |
| shophouse |  540000000.0   |
| shophouse |  890000000.0   |
| shopho